In [1]:

from helper import signal_utils
import soundfile as sf
import os
import shutil

import matplotlib
matplotlib.use('Agg')



from torchvision import transforms
import torch
from torch.utils.data import DataLoader
from torchvision import models


import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from helper import train_utils




input_wavefiles_path='dataset/wavefiles'
output_graphs_path = 'dataset/graphs'
feature_path = 'dataset/features'


from timeit import default_timer as timer

s_time = timer()

N_FFTs = [1024*(i+1) for i in range(8)]


label_csv_file = 'dataset/labels.csv'
# path_all_graphs='dataset/graphs'#'dataset/challenge_wav/afew_output/X_train_plots'

batch_sizes = [16, 32, 64, 128, 256]
epochs = 1

transform =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
])


if os.path.exists(f'{feature_path}'):
    shutil.rmtree(f'{feature_path}')
os.mkdir(f'{feature_path}')



for N_FFT in N_FFTs:
    if os.path.exists(f'{output_graphs_path}'):
        shutil.rmtree(f'{output_graphs_path}')
    os.mkdir(f'{output_graphs_path}')

    print('Graph generating...')
    for file in os.listdir(input_wavefiles_path):
        signal_array, samplig_rate = sf.read(f'{input_wavefiles_path}/{file}')
        fig, _ = signal_utils.melspectrogram_db(signal_array=signal_array, samplig_rate=samplig_rate, fmax= samplig_rate//2,
                                                out_spec_path=f'{output_graphs_path}/{str(file).split(".")[0]}.jpg', n_mels=90,
                                                hop_length=512, n_fft=N_FFT,  cmap = 'jet', axis_off = True)
    print(timer() - s_time )

    #######################################



    for batch_size in batch_sizes:
        experience_name = f'Melspectogram_Experiment_N_FFT_{N_FFT}_batch_size_{batch_size}'
        
        print(f"Preparing data as: {experience_name}")

        # Split the src trainset into train and test
        
        src_data_dir = f'{output_graphs_path}'
        dest_data_dir =f'dataset/graph_classes_{N_FFT}_batch_size_{batch_size}'
        n_classes =  train_utils.prepare_data_classes_from_train(src_data_dir, dest_data_dir, label_csv_file, move=False)

        ################################

        X_train, X_test = train_utils.split_train(dest_data_dir, 0.8, transform)
        train_dataloader = DataLoader(X_train, batch_size=batch_size, shuffle=True)
        test_dataloader = DataLoader(X_test, batch_size=batch_size, shuffle=True)

        ################################

        model = models.vgg16(pretrained=True)
        for param in model.parameters():
                param.requires_grad = False
        if train_utils.train_on_gpu:
            model = model.to('cuda')

        ################################

        print(f"Train_set feature extracting using pre-trained VGG16 for {experience_name} ...")

        y_true = []#torch.tensor(())#[]
        outputs = []#torch.tensor(())#[]
        l = len(train_dataloader)
        start = timer()

        for i, (inputs, labels) in enumerate(train_dataloader):
                output = model(inputs) # Feed Network
                outputs.append(output)
                y_true.append(labels.data)
                train_utils.printProgressBar(i + 1, l, prefix = f'{experience_name}: ', suffix = f'{timer() - start:.2f} sec.', length = 50)            

        outputs_ = torch.cat(outputs, dim=0) 
        y_true_ = torch.cat(y_true, dim=0) 
        new_data = torch.utils.data.TensorDataset(outputs_, y_true_)
        torch.save(new_data,f'{feature_path}/train_features_{experience_name}')

        print(f"Test_set feature extracting using pre-trained VGG16 for {experience_name} ...")

        y_true = []#torch.tensor(())#[]
        outputs = []#torch.tensor(())#[]
        l = len(test_dataloader)
        start = timer()

        for i, (inputs, labels) in enumerate(test_dataloader):
                output = model(inputs) # Feed Network
                outputs.append(output)
                y_true.append(labels.data)
                train_utils.printProgressBar(i + 1, l, prefix = f'{experience_name}: ', suffix = f'{timer() - start:.2f} sec.', length = 50)            


        outputs_ = torch.cat(outputs, dim=0) 
        y_true_ = torch.cat(y_true, dim=0) 
        new_data = torch.utils.data.TensorDataset(outputs_, y_true_)
        torch.save(new_data,f'{feature_path}/test_features_{experience_name}')
        print('=====================================')
    print('=====================================')
print('=====================================')


Train on gpu: False
Graph generating...


/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


93.34584936501051
Preparing data as: Melspectogram_Experiment_N_FFT_1024_batch_size_16
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_1024_batch_size_16 ...
Melspectogram_Experiment_N_FFT_1024_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 139.10 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_1024_batch_size_16 ...
Melspectogram_Experiment_N_FFT_1024_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 35.50 sec.
Preparing data as: Melspectogram_Experiment_N_FFT_1024_batch_size_32
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_1024_batch_size_32 ...
Melspectogram_Experiment_N_FFT_1024_batch_size_32:  |██████████████████████████████████████████████████| 100.0% 138.42 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_1024_batch_size_32 ...
Melspectogram_Experiment_N_FFT_

/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1600
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=2000
  return f(*args, **kwargs)


1102.556379267
Preparing data as: Melspectogram_Experiment_N_FFT_2048_batch_size_16
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_2048_batch_size_16 ...
Melspectogram_Experiment_N_FFT_2048_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 153.45 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_2048_batch_size_16 ...
Melspectogram_Experiment_N_FFT_2048_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 37.96 sec.
Preparing data as: Melspectogram_Experiment_N_FFT_2048_batch_size_32
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_2048_batch_size_32 ...
Melspectogram_Experiment_N_FFT_2048_batch_size_32:  |██████████████████████████████████████████████████| 100.0% 153.49 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_2048_batch_size_32 ...
Melspectogram_Experiment_N_FFT_204

...

5655.801223896007
Preparing data as: Melspectogram_Experiment_N_FFT_6144_batch_size_16
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_6144_batch_size_16 ...
Melspectogram_Experiment_N_FFT_6144_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 155.64 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_6144_batch_size_16 ...
Melspectogram_Experiment_N_FFT_6144_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 43.04 sec.
Preparing data as: Melspectogram_Experiment_N_FFT_6144_batch_size_32
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_6144_batch_size_32 ...
Melspectogram_Experiment_N_FFT_6144_batch_size_32:  |██████████████████████████████████████████████████| 100.0% 172.74 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_6144_batch_size_32 ...
Melspectogram_Experiment_N_FFT_

/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=7168 is too small for input signal of length=3026
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=7168 is too small for input signal of length=3364
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=7168 is too small for input signal of length=4000
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=7168 is too small for input signal of length=1600
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=716

6765.303700254997
Preparing data as: Melspectogram_Experiment_N_FFT_7168_batch_size_16
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_7168_batch_size_16 ...
Melspectogram_Experiment_N_FFT_7168_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 122.36 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_7168_batch_size_16 ...
Melspectogram_Experiment_N_FFT_7168_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 24.81 sec.
Preparing data as: Melspectogram_Experiment_N_FFT_7168_batch_size_32
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_7168_batch_size_32 ...
Melspectogram_Experiment_N_FFT_7168_batch_size_32:  |██████████████████████████████████████████████████| 100.0% 117.31 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_7168_batch_size_32 ...
Melspectogram_Experiment_N_FFT_

/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=8192 is too small for input signal of length=7168
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=8192 is too small for input signal of length=3026
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=8192 is too small for input signal of length=3364
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=8192 is too small for input signal of length=4000
  return f(*args, **kwargs)
/home/mazdak/projects/univ-lille/snnSims/snnTorch/snnTorch/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=819

7587.862039077998
Preparing data as: Melspectogram_Experiment_N_FFT_8192_batch_size_16
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_8192_batch_size_16 ...
Melspectogram_Experiment_N_FFT_8192_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 107.41 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_8192_batch_size_16 ...
Melspectogram_Experiment_N_FFT_8192_batch_size_16:  |██████████████████████████████████████████████████| 100.0% 39.41 sec.
Preparing data as: Melspectogram_Experiment_N_FFT_8192_batch_size_32
Train_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_8192_batch_size_32 ...
Melspectogram_Experiment_N_FFT_8192_batch_size_32:  |██████████████████████████████████████████████████| 100.0% 122.64 sec.
Test_set feature extracting using pre-trained VGG16 for Melspectogram_Experiment_N_FFT_8192_batch_size_32 ...
Melspectogram_Experiment_N_FFT_

## file sizes
## batch size effect on the same N_FFT
## compare the result of using only the features with a small network or using entire the network (time/energy/acc/scores ...). Does accuracy worth such an amount of energy?